In [11]:
import pandas as pd
from openpyxl import load_workbook

def process_race_data(input_file_path, output_file_path):
    # Read the Excel file
    df = pd.read_excel(input_file_path)

    # Sort the data by RunnerID and DateOfTheRace
    df.sort_values(by=['RunnerID', 'DateOfTheRace'], ascending=[True, True], inplace=True)

    # Create a new DataFrame to store the results
    output_data = pd.DataFrame(columns=['RunnerID', 'Marathon', 'MarathonResult', 
                                        'RaceOne', 'RaceOneResult', 'RaceOneKM', 
                                        'RaceTwo', 'RaceTwoResult', 'RaceTwoKM'])

    # Iterate over unique RunnerIDs
    for runner_id in df['RunnerID'].unique():
        runner_data = df[df['RunnerID'] == runner_id]

        # Find the marathon (race with distance > 40000 meters)
        marathon_data = runner_data[runner_data['NumberOfMeters'] > 40000].tail(1)

        if marathon_data.empty:
            continue

        marathon_distance = marathon_data['NumberOfMeters'].values[0]
        marathon_result = marathon_data['Result'].values[0]

        # Find the races before the marathon
        races_before_marathon = runner_data[runner_data['DateOfTheRace'] < marathon_data['DateOfTheRace'].values[0]]

        if len(races_before_marathon) >= 2:
            race_one = races_before_marathon.tail(1)
            race_two = races_before_marathon.tail(2).head(1)

            race_one_name = race_one['NameOfTheRace'].values[0]
            race_one_result = race_one['Result'].values[0]
            race_one_distance = race_one['NumberOfMeters'].values[0]

            race_two_name = race_two['NameOfTheRace'].values[0]
            race_two_result = race_two['Result'].values[0]
            race_two_distance = race_two['NumberOfMeters'].values[0]
        else:
            # If there are no two races before the marathon, skip this runner
            continue

        output_data = pd.concat([output_data, pd.DataFrame([{
            'RunnerID': runner_id,
            'Marathon': marathon_distance,
            'MarathonResult': marathon_result,
            'RaceOne': race_one_name,
            'RaceOneResult': race_one_result,
            'RaceOneKM': race_one_distance,
            'RaceTwo': race_two_name,
            'RaceTwoResult': race_two_result,
            'RaceTwoKM': race_two_distance
        }])], ignore_index=True)
    output_data.to_csv(output_file_path, index=False)
    # Save the output data to a new sheet in the same Excel file
    #book = load_workbook(output_file_path)
    #with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    #    writer.book = book
     #   writer.sheets.update(dict((ws.title, ws) for ws in book.worksheets))
    #    output_data.to_excel(writer, sheet_name='Processed_Data', index=False)
    #writer.save()
    #writer.close()

# Example usage:
input_file_path = 'InputToProcess.xlsx'
output_file_path = 'OutputFromProcess.csv'
process_race_data(input_file_path, output_file_path)


Exception ignored in: <function ZipFile.__del__ at 0x7fb3a6ab56c0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/zipfile.py", line 1874, in __del__
    self.close()
  File "/opt/conda/lib/python3.11/zipfile.py", line 1891, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
